In [13]:
from dash import Dash,html,dcc,Input,Output,callback
import pandas as pd

app = Dash(__name__)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


In [14]:
display(df)
dff = df[df['Year'] == 1962]
dff[dff['Indicator Name'] == 'Agriculture, value added (% of GDP)']['Value']
dff[dff['Indicator Name'] == 'Agriculture, value added (% of GDP)']['Country Name']


,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


0                           Arab World
14              Caribbean small states
28      Central Europe and the Baltics
42          Early-demographic dividend
56                 East Asia & Pacific
                     ...              
3626                    Virgin Islands
3640                West Bank and Gaza
3654                       Yemen, Rep.
3668                            Zambia
3682                          Zimbabwe
Name: Country Name, Length: 264, dtype: object

In [15]:
import plotly.express as px

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear','Log'],
                'Linear',
                id = 'xaxis-type',
                inline=True
            )
        ],style={'width':'48%','display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear','Log'],
                'Linear',
                id = 'yaxis-type',
                inline=True
            )
        ],style={'width':'48%','float':'right','display':'inline-block'})
    ]),
    dcc.Graph(id='indicator-graphic'),
    dcc.Slider(
        min=df['Year'].min(),
        max=df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks = {str(year):str(year) for year in df['Year'].unique()}
    )

])

@callback(
    Output('indicator-graphic','figure'),
    Input('xaxis-column','value'),
    Input('yaxis-column','value'),
    Input('xaxis-type','value'),
    Input('yaxis-type','value'),
    Input('year--slider','value')   
)
def update_graph(xaxis_column_name,
                 yaxis_column_name,
                 xaxis_type,
                 yaxis_type,
                 year_value
                 ):
    dff = df[df['Year'] == year_value]
    xValue = dff[dff['Indicator Name'] == xaxis_column_name]['Value']
    yValue = dff[dff['Indicator Name'] == yaxis_column_name]['Value']
    hoverValue = dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
    fig = px.scatter(
                x=xValue,
                y=yValue,
                hover_name=hoverValue)
    
    fig.update_layout(margin={'l':40, 'b':40, 't':10, 'r':0},
                      hovermode='closest')
    
    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')
    
    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')
    return fig

    

if __name__ == '__main__':
    app.run(debug=True)